In [91]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [92]:
train_datagen = ImageDataGenerator (rescale=1./255, shear_range=0.1,zoom_range=0.1, horizontal_flip=True)

In [93]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [94]:
training_set = train_datagen.flow_from_directory("C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\body\\training",target_size = (224, 224),batch_size = 10,class_mode= 'categorical')
test_set = test_datagen.flow_from_directory("C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\body\\validation",target_size = (224, 224),batch_size = 10,class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [95]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [96]:
imageSize=[224,224]
Vgg = VGG16(input_shape=imageSize+[3], weights='imagenet', include_top=False)

In [97]:
for layer in Vgg.layers:
    layer.trainable =False

In [98]:
X = Flatten()(Vgg.output)

In [99]:
prediction = Dense(3, activation='softmax')(X)

In [102]:
model=Model(inputs=Vgg.input,outputs=prediction)

In [103]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['acc'])

In [104]:
import sys
#fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=979//10,
  validation_steps=171//10)

C:\Users\ssdha\AppData\Local\Temp\ipykernel_15632\1270631010.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
97/97 [==============================] - 644s 7s/step - loss: 1.2665 - acc: 0.5315 - val_loss: 0.8390 - val_acc: 0.6765
Epoch 2/25
97/97 [==============================] - 447s 5s/step - loss: 0.7126 - acc: 0.7162 - val_loss: 0.9837 - val_acc: 0.6412
Epoch 3/25
97/97 [==============================] - 469s 5s/step - loss: 0.4864 - acc: 0.8070 - val_loss: 0.9016 - val_acc: 0.6765
Epoch 4/25
97/97 [==============================] - 474s 5s/step - loss: 0.3601 - acc: 0.8607 - val_loss: 1.0623 - val_acc: 0.6647
Epoch 5/25
97/97 [==============================] - 474s 5s/step - loss: 0.2620 - acc: 0.9112 - val_loss: 0.9379 - val_acc: 0.6529
Epoch 6/25
97/97 [==============================] - 461s 5s/step - loss: 0.2063 - acc: 0.9267 - val_loss: 0.9162 - val_acc: 0.7000
Epoch 7/25
97/97 [==============================] - 484s 5s/step - loss: 0.1723 - acc: 0.9474 - val_loss: 0.9836 - val_acc: 0.6588
Epoch 8/25
97/97 [==============================] - 475s 5s/step - loss: 0.1129 - a

In [105]:
model.save('body.h5')

In [106]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [107]:
model=load_model('body.h5')

In [134]:
#creating a function, which is used to predict, so need some preprocessing 
def detect (frame):
    img = cv2.resize(frame, (224,224)) #resizing the image to model trained image size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # uploaded image is in the form of BGR, so convert to RGB
    # scaling need to be done
    if(np.max(img)>1):
        img=img/255.0
    img= np.array([img]) #then to array fpormat 
    prediction = model.predict(img)
    print(prediction)
    label=["front", "rear","side"]
    preds =label [np.argmax(prediction)]
    return preds

In [135]:
import numpy as np

In [1]:
data="C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\body\\validation\\01-rear\\0019.JPEG"
image=cv2.imread(data)
print(detect(image))

NameError: name 'cv2' is not defined

1/1 [==============================] - 0s 184ms/step
[[3.996842e-02 9.600312e-01 3.556172e-07]]
rear
